#  Setup stuff: Connectivity

In [ ]:

#  Setting display options

from tabulate import tabulate
import pandas as pd
   #
pd.set_option("display.width", 480)

print("--")


In [ ]:

from katana import remote
from katana.remote import import_data


my_client = remote.Client()

print(my_client)


In [ ]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


In [ ]:
#  DELETE ALL GRAPHS

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name=l_database.name).graphs_in_database():
      l_handle=my_client.get_database(name=l_database.name).get_graph_by_id(id=l_graph.graph_id)
      l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "      GRAPH Version: ", l_graph.version)

print("--")


In [ ]:
#  DELETE ALL DATABASES

for l_database in my_client.databases():
   if (l_database.name != "default"):
      my_client.get_database(name=l_database.name).delete_database()
      print("--")

for l_database in my_client.databases():
   print("DB ID: ", l_database.database_id, "     DB Name: ", l_database.name)


In [ ]:
#  CREATE DATABASE

my_database = my_client.create_database(name=DB_NAME)

print(my_database.database_id)


In [ ]:
#  CREATE GRAPH
#
my_graph = my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=NUM_PARTITIONS)

print(my_graph)


# Create: Vertices/nodes, edges ..

In [ ]:

#  Vertices/Nodes, Airports ..

df_airports = pd.DataFrame([
       #
   ["MKE", "Milwaukee"               , "Airport"],
   ["ORD", "Chicago O-Hare"          , "Airport"],
   ["SJC", "San Jose"                , "Airport"],
   ["DEN", "Denver"                  , "Airport"],
       #
   ], columns = ["airport_code", "airport_name", "LABEL"])

print(tabulate(df_airports, headers='keys', tablefmt='psql'))


In [ ]:
#  Create the edge, flights ..

df_flights = pd.DataFrame([
      #
   ["MKE", "ORD",   66, 1, "FLIES_TO" ],
   ["ORD", "MKE",   66, 1, "FLIES_TO" ],
      #
   ["ORD", "DEN",  886, 1, "FLIES_TO" ],
   ["DEN", "ORD",  886, 1, "FLIES_TO" ],
      #
   ["SJC", "DEN",  948, 1, "FLIES_TO" ],                           #  Notice SJC flies to/from Denver
   ["DEN", "SJC",  948, 1, "FLIES_TO" ],
      #
   ["SJC", "ORD", 1829, 1, "FLIES_TO" ],                           #  Notice SJC flies to ORD, but not ORD to SJC
      #
   ], columns = ["START_ID", "END_ID", "DISTANCE", "NUM_HOPS", "TYPE"])

print(tabulate(df_flights, headers='keys', tablefmt='psql'))


In [ ]:

# Import the 2 previously created Python DataFrames into KatanaGraph ..

with import_data.DataFrameImporter(my_graph) as df_importer:   
    
   df_importer.nodes_dataframe(df_airports,                    #  Aiports set of Nodes
      id_column             = "airport_code",
      id_space              = "Airport",  
      label                 = "Airport",  
      )
   
   df_importer.edges_dataframe(df_flights,                     #  Our Edge, specifying the relationship between Airport --> FLIES_TO --> Airport
      source_id_space       = "Airport", 
      destination_id_space  = "Airport",   
      source_column         = "START_ID",
      destination_column    = "END_ID",
      type                  = "FLIES_TO"
      )
   
   df_importer.insert()

print("--")


#  Actual analytics ..

In [ ]:

from katana import remote
from katana.remote import analytics


l_cntr = 20

print("--")


In [29]:

l_result1 = my_graph.query(
   """
   MATCH (n: Airport) 
   WHERE n.airport_code = 'MKE'
   RETURN n
   """ )
for l_each in l_result1.itertuples():
   print(l_each[1])

l_key = 


          0/? [?op/s]

562949953421312


# Iteration/test 1 ..

In [ ]:

#  Running analytics.sssp() .. iteration 1 ..
#
#  This used to work, and it works no longer. See comments after this cell.
#

l_source_nodeid       = "2"                        #  id of node to start on (this is SJC)
   #
l_weight_property     = "DISTANCE"                 #  Edge property

l_max_iterations      = 10000 


#  Generate a unique prop name so we can run this over and over without edit ..
#
l_cntr += 1
   #
l_output_propname     = "sssp_dist_" + str(l_cntr).zfill(4)


analytics.sssp(
   my_graph,
      #
   source               = l_source_nodeid,
      #
   weight_property      = l_weight_property,
   max_iterations       = l_max_iterations,
   result_property_name = l_output_propname,
   )

display(print("--"))


In [ ]:

l_result1 = my_graph.query(
   """
   MATCH (n) 
   RETURN n
   """ )
print(tabulate(l_result1, headers='keys', tablefmt='psql'))

l_result2 = my_graph.query(
   """
   MATCH (n) - [r] -> (m)
   RETURN r
   """ )
print(tabulate(l_result2, headers='keys', tablefmt='psql'))


#  Sample output,
#
#     +----+-----------------+-------------+-----------+------------------+--------+----------+
#     |    |   n.internal_id | n.labels    | n.LABEL   | n.airport_name   | n.id   | n.type   |
#     |----+-----------------+-------------+-----------+------------------+--------+----------|
#     |  0 |               0 | ['Airport'] | Airport   | Milwaukee        | MKE    | node     |
#     |  1 |               1 | ['Airport'] | Airport   | Denver           | DEN    | node     |
#     |  2 | 281474976710656 | ['Airport'] | Airport   | San Jose         | SJC    | node     |
#     |  3 | 562949953421312 | ['Airport'] | Airport   | Chicago O-Hare   | ORD    | node     |
#     +----+-----------------+-------------+-----------+------------------+--------+----------+
#     
#     +----+-----------------+----------------+-----------------+-----------+--------------+--------------+----------+-----------------+------------------+----------+
#     |    |        r.end.id | r.end.labels   |   r.internal_id | r.label   |   r.DISTANCE |   r.NUM_HOPS | r.TYPE   |      r.start.id | r.start.labels   | r.type   |
#     |----+-----------------+----------------+-----------------+-----------+--------------+--------------+----------+-----------------+------------------+----------|
#     |  0 | 281474976710656 | ['Airport']    |               0 | FLIES_TO  |          948 |            1 | FLIES_TO |               1 | ['Airport']      | edge     |
#     |  1 | 562949953421312 | ['Airport']    |               2 | FLIES_TO  |          886 |            1 | FLIES_TO |               1 | ['Airport']      | edge     |
#     |  2 | 562949953421312 | ['Airport']    |               1 | FLIES_TO  |           66 |            1 | FLIES_TO |               0 | ['Airport']      | edge     |
#     |  3 | 562949953421312 | ['Airport']    | 281474976710656 | FLIES_TO  |         1829 |            1 | FLIES_TO | 281474976710656 | ['Airport']      | edge     |
#     |  4 |               1 | ['Airport']    | 281474976710657 | FLIES_TO  |          948 |            1 | FLIES_TO | 281474976710656 | ['Airport']      | edge     |
#     |  5 |               1 | ['Airport']    | 562949953421312 | FLIES_TO  |          886 |            1 | FLIES_TO | 562949953421312 | ['Airport']      | edge     |
#     |  6 |               0 | ['Airport']    | 562949953421313 | FLIES_TO  |           66 |            1 | FLIES_TO | 562949953421312 | ['Airport']      | edge     |
#     +----+-----------------+----------------+-----------------+-----------+--------------+--------------+----------+-----------------+------------------+----------+


#  Results,
#
#     .  This block used to work.
#        The "id" for nodes was supplied as a unique character string, and our DataFrame importer would
#        overwrite that value with an integer.
#        Apparently that behavior has changed.
#        Okay. Good
#
#     .  What we received is that the source (source node id) was not found, so no result_property_name
#        was output/created.
#
#         Should an error have been output instead ??


# Iteration/test 2 ..

In [ ]:

#  Running analytics.sssp() .. iteration 2 ..
#
#  Change source (source node id) to "SJC"
#

l_source_nodeid       = "SJC"                      #  id of node to start on (this is SJC)
   #
l_weight_property     = "DISTANCE"                 #  Edge property

l_max_iterations      = 10000 


#  Generate a unique prop name so we can run this over and over without edit ..
#
l_cntr += 1
   #
l_output_propname     = "sssp_dist_" + str(l_cntr).zfill(4)


analytics.sssp(
   my_graph,
      #
   source               = l_source_nodeid,
      #
   weight_property      = l_weight_property,
   max_iterations       = l_max_iterations,
   result_property_name = l_output_propname,
   )

display(print("--"))

#  Error received,
#
#     InvalidRequestError: b'{"code":400,"message":"json: invalid uint64 value; should be encoded as a string"}'

